In [150]:
%run "..\general_functions\generalFunctions.ipynb"
%run "..\Promotion Slide Duplicate\Promotion Replacement FunctionNEW.ipynb"

## Parameters

In [ ]:
normalized = True
promo_type = False
display_share = True  # True if Available
feature_share = True

In [152]:
ManufOrTopC ="Top Companies"
BrandOrTopB = "Top Brands"
prodORitem="Business Name"

client_manuf = ["Bel"]
client_brands = ["Kiri", "La Vache Qui Rit", "Boursin"]

categories = ["Total Fromage"]
sectors = ["Soft Cheese", "Aperitif"]
segments = ["Enfant", "Frais A Tartiner", "Salade"]
subsegments= []
subcategories= []

decimals = 2
sign = "After"
currency = '€'
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '

customareas=''
national = False
areas = ["RETAILER"]
regions_RET  =["Carrefour", "Intermarche"]
channels_RET = ["Carrefour Hyper + Drive", "Carrefour Supermarket + Drive", "Carrefour Proximite", "Intermarche Super", "Intermarche Hyper", "Intermarche Proxi"]
market_RET = []

regions_CHAN = []
channels_CHAN = []
market_CHAN = []

regions_CUST = []
channels_CUST = []
market_CUST = []

data_source = "DATA SOURCE: Trade Panel/Retailer Data | July 2025"
years = ['2023', '2024', '2025']
slides_Period="P3M"

subcatg_parent = "Segment"
subcatg_parent_list = segments
 
percent = 1000000
percentstr="'000 000"
start_date = "2022-08-01"	
end_date = "2025-08-01"

promo_col = []
selectedBrands = client_brands 
marketList = regions_RET + channels_RET + market_RET + regions_CHAN + channels_CHAN + market_CHAN 
notInScope = []
OpenEditData=True
direct_parent = {"Sector":"Category",
                "Segment":"Sector",
                "SubSegment":"Segment", 
                "SubCategory":"Segment"}

In [153]:
marketList = regions_RET + channels_RET + market_RET + regions_CHAN + channels_CHAN + market_CHAN + regions_CUST + channels_CUST + market_CUST
categoryList=categories +sectors+segments+subsegments+subcategories
defaults = {
    'Aperitif': 0.27,
    'Enfant': 0.35,
    "Frais A Tartiner" : 0.25,
    "Ingredient A Chaud" : 0.3,
    "Salade" : 0.28
}
diff_market_value = {

}

def totalsize (lis,defaultdic,diffmarketdic=[]):

    max_total_size = {
    f"{category} | {market}": diff_market_value.get(market.upper(), {}).get(category, defaults[category])
    for market in lis
    for category in defaults
    }

    return max_total_size  

max_total_size=totalsize(marketList,defaults,diff_market_value)

custom_colors = [
    RGBColor(91, 159, 153),    # Darker teal
    RGBColor(131, 199, 193),   # Brighter medium teal
    RGBColor(168, 216, 212),   # Original light teal
    RGBColor(198, 236, 232),   # Very light teal
    RGBColor(111, 179, 173),
    RGBColor(121, 189, 183)
]
# custom_colors = [
#     RGBColor(111, 179, 173),  
#     RGBColor(121, 189, 183),  
#     RGBColor(168, 216, 212),  
#     RGBColor(178, 226, 222),
# ]
 

In [154]:

Scope = {
    "Category": categories,
    "Sector": sectors,
    "Segment": segments,
    "Subsegment": subsegments,
    "Subcategory": subcategories
}
suffixes = ["Category", "Sector", "Segment",'SubSegment', 'SubCategory']

## Reading datasets

In [155]:
loaded_data = {}
datasets_path = os.getcwd()+"/Promotion Datasets NewEX - P3M/"
datasets = os.listdir(datasets_path)
for d in datasets:
    with open(datasets_path+d, 'rb') as handle:
        globals()[d.split('.')[0]] = pd.read_pickle(handle)

## Data Cleaning

In [156]:
globals()[f"modified_promotionBrands{slides_Period}"] = cleaningData(globals()[f"promotions_brands_{slides_Period}"])

In [157]:
if feature_share & display_share:
    globals()[f"modified_promotionBrands_share{slides_Period}"] = cleaningData_featureshare(globals()[f"promotions_brands_{slides_Period}"])

In [158]:
globals()[f"modified_promotionBrands{slides_Period}_total"] = cleaningdata_with_grand_total(globals()[f"promotions_brands_{slides_Period}"])

In [159]:
globals()[f"modified_promotionProducts{slides_Period}"] = cleaningData(globals()[f"promotions_products_{slides_Period}"])

In [160]:
globals()[f"modified_promotionProducts{slides_Period}_updated"] = {}
for key, df in globals()[f"modified_promotionProducts{slides_Period}"].items():
    df = df.copy()
    df = df[df[f'{prodORitem}'] != '']
    df = df[df['Promo Sales'] >= 10000]
    df = df.sort_values(by='Promo Value', ascending=False).reset_index(drop=True)
    if not df.empty:
        globals()[f"modified_promotionProducts{slides_Period}_updated"][key] = df
globals()[f"modified_promotionProducts{slides_Period}_volumeuplift"] = globals()[f"modified_promotionProducts{slides_Period}_updated"]

In [161]:
modified_promotionEndOfWeek = cleaningData(promotions_EndOfWeek)

In [162]:
modified_valueUplift = cleaningData(value_uplift)

In [163]:
newModifiedBrands = cleaning13New(globals()[f"promotions_brands_{slides_Period}"],1)

In [164]:
# Define parameters for each pivot call in a list of tuples:
pivot_params = [
    (Sector_client_VSOD, 'Top Brands', 'Sector_client_VSODNew'),
    (Sector_manuf_VSOD, 'Top Companies', 'Sector_manuf_VSODNew'),
    (Segment_client_VSOD, 'Top Brands', 'Segment_client_VSODNew'),
    (Segment_manuf_VSOD, 'Top Companies', 'Segment_manuf_VSODNew'),
    (SubSegment_client_VSOD, 'Top Brands', 'SubSegment_client_VSODNew'),
    (SubSegment_manuf_VSOD, 'Top Companies', 'SubSegment_manuf_VSODNew'),
    (SubCategory_client_VSOD, 'Top Brands', 'SubCategory_client_VSODNew'),
    (SubCategory_manuf_VSOD, 'Top Companies', 'SubCategory_manuf_VSODNew')
]

# Prepare a dictionary to store results:
pivot_results = {}

for data_dict, pivot_col, result_name in pivot_params:
    pivot_results[result_name] = dict_to_pivot_general(
        data_dict=data_dict,
        pivot_col=pivot_col,
        value_col='VSOD',
        aggfunc='sum',
        fill_value=pd.NA
    )

# If you want to have these as separate variables in your workspace, you can unpack like:
Sector_client_VSODNew = pivot_results['Sector_client_VSODNew']
Sector_manuf_VSODNew = pivot_results['Sector_manuf_VSODNew']
Segment_client_VSODNew = pivot_results['Segment_client_VSODNew']
Segment_manuf_VSODNew = pivot_results['Segment_manuf_VSODNew']
SubSegment_client_VSODNew = pivot_results['SubSegment_client_VSODNew']
SubSegment_manuf_VSODNew = pivot_results['SubSegment_manuf_VSODNew']
SubCategory_client_VSODNew = pivot_results['SubCategory_client_VSODNew']
SubCategory_manuf_VSODNew = pivot_results['SubCategory_manuf_VSODNew']


In [165]:
if len(sectors) >0:
    a = Sector_VSOD
    if len(Sector_client_VSODNew) >0:
        b = cleaningData(Sector_client_VSODNew)
        sect_vsod_merged = merging(b,a, col=[direct_parent["Sector"],'Sector'])
    else:
        sect_vsod_merged = a
    c = cleaningData(Sector_manuf_VSODNew)  
    for key in sect_vsod_merged:
        merged_df = pd.merge(sect_vsod_merged[key], c[key], on=[direct_parent["Sector"],'Sector'], how='left')
        if merged_df.shape[0]>0:
            sect_vsod_merged[key] = merged_df    

if len(segments) >0:
    a = Segment_VSOD
    if len(Segment_client_VSODNew) > 0:
        b = cleaningData(Segment_client_VSODNew)
        seg_vsod_merged = merging(a,b, col=[direct_parent["Segment"],'Segment'])
    else:
        seg_vsod_merged = a
    
    c = cleaningData(Segment_manuf_VSODNew)
    for key in seg_vsod_merged:
        # Merge DataFrames based on 'Sector' column
        merged_df = pd.merge(seg_vsod_merged[key], c[key], on=[direct_parent["Segment"],'Segment'], how='left')
        merged_df = merged_df.fillna(0)
        if merged_df.shape[0]>0:
            seg_vsod_merged[key] = merged_df    

if len(subsegments) >0:
    a = SubSegment_VSOD
    if len(SubSegment_client_VSODNew) > 0 :
        b = cleaningData(SubSegment_client_VSODNew)
        subseg_vsod_merged = merging(a,b, col=[direct_parent["SubSegment"],'SubSegment'])
    else:
        subseg_vsod_merged = a
    c = cleaningData(SubSegment_manuf_VSOD)
    for key in subseg_vsod_merged:
        # Merge DataFrames based on 'Sector' column
        merged_df = pd.merge(subseg_vsod_merged[key], c[key], on=[direct_parent["SubSegment"],'SubSegment'], how='left')
        merged_df = merged_df.fillna(0)
        if merged_df.shape[0]>0:
            subseg_vsod_merged[key] = merged_df    

if len(subcategories) >0:
    a = SubCategory_VSOD
    if len(SubCategory_client_VSODNew) > 0 :
        b = cleaningData(SubCategory_client_VSODNew)
        subcat_vsod_merged = merging(a,b, col=[direct_parent["SubCategory"],'SubCategory'])
    else:
        subcat_vsod_merged = a
    c = cleaningData(SubCategory_manuf_VSODNew)
    for key in subcat_vsod_merged:
        # Merge DataFrames based on 'Sector' column
        merged_df = pd.merge(subcat_vsod_merged[key], c[key], on=[direct_parent["SubCategory"],'SubCategory'], how='left')
        merged_df = merged_df.fillna(0)
        if merged_df.shape[0]>0:
            subcat_vsod_merged[key] = merged_df  

In [166]:
client=client_brands+client_manuf
if len(sectors)!=0:
    sect_vsod_merged=splitkeys(sect_vsod_merged,categories,parent=direct_parent['Sector'],clientlist=client)
if len(segments)!=0:
    seg_vsod_merged=splitkeys(seg_vsod_merged,sectors,parent=direct_parent['Segment'],clientlist=client)
if len(subsegments)!=0:
    subseg_vsod_merged=splitkeys(subseg_vsod_merged,segments,parent=direct_parent['SubSegment'],clientlist=client)
if len(subcategories)!=0:
    subcat_vsod_merged=splitkeys(subcat_vsod_merged,segments,parent=direct_parent['SubCategory'],clientlist=client)

In [167]:
if len(sectors):
    sect_vsod_count =0
    for key,df in sect_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                sect_vsod_count +=1
    sect_vsod_count = sect_vsod_count *len(categories)
 
if len(segments):
    seg_vsod_count =0
    for key,df in seg_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                seg_vsod_count +=1
    #seg_vsod_count = seg_vsod_count * len(sectors) 
    seg_vsod_count = seg_vsod_count           
 
if len(subsegments) >0:
    subseg_vsod_count =0
    for key,df in subseg_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                subseg_vsod_count +=1
    #subseg_vsod_count =subseg_vsod_count *len(segments)
    subseg_vsod_count = subseg_vsod_count
 
if len(subcategories) >0:
    subcat_vsod_count =0
    for key,df in subcat_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                subcat_vsod_count +=1
    #subcat_vsod_count = subcat_vsod_count * len(subsegments)
    subcat_vsod_count = subcat_vsod_count

 

In [168]:
promotionsBrandSortedTotalFinal={}
promotionsBrandSortedTotal=dfSort(globals()[f"modified_promotionBrands{slides_Period}"], client_brands, "Top Brands", num=8,salesCol='Promo Value')
for key,df in promotionsBrandSortedTotal.items():
     df_client = selectClientBrands(promotionsBrandSortedTotal[key],'Top Brands', 'Promo Value')
     number_of_brands_needed = max(6 - len(df_client),0)
     
     df = df[~df['Top Brands'].isin(client_brands)]
     df = df[~df['Top Brands'].str.contains('Others', case=False)]
     
     df = df.sort_values(by='Promo Value', ascending=False).head(number_of_brands_needed)
     df = pd.concat([df, df_client], ignore_index=True)
     df = df.sort_values(by='Promo Value', ascending=False).reset_index(drop=True)
     df = df[~df['Top Brands'].str.contains('Grand Total', case=False)]
     df = df[df['Value Share'] > 0.01]
        
     df['VSOD Evaluation vs YA'] = df['VSOD Evaluation vs YA'].astype(float)
     df['Promo Value Uplift vs YA'] = df['Promo Value Uplift vs YA'].astype(float)
     
     if df.shape[0] >0:
          promotionsBrandSortedTotalFinal[key] = df

In [169]:
promotionsBrandNOTSortedTotalFinal={}
promotionsBrandNOTSortedTotalFinal=dfSort(globals()[f"modified_promotionBrands{slides_Period}"], client_brands, "Top Brands", num=8,salesCol='Promo Value')
for key,df in globals()[f"modified_promotionBrands{slides_Period}"].items():
     df = df.sort_values(by='Promo Value', ascending=False).reset_index(drop=True)
     df = df[~df['Top Brands'].str.contains('Others', case=False)]
     df = df[~df['Top Brands'].str.contains('Grand Total', case=False)]
     df = df[df['Value Share'] > 0.01]
     df['VSOD Evaluation vs YA'] = df['VSOD Evaluation vs YA'].astype(float)
     df['Promo Value Uplift vs YA'] = df['Promo Value Uplift vs YA'].astype(float)
     if df.shape[0] >0:
          promotionsBrandNOTSortedTotalFinal[key] = df

In [170]:
selectedBrands_og = selectedBrands
selectedBrands= selectedBrands + ["Grand Total"]

In [171]:
promotionsBrandsSelected={key:globals()[f"modified_promotionBrands{slides_Period}_total"][key][globals()[f"modified_promotionBrands{slides_Period}_total"][key]['Top Brands'].isin(selectedBrands)].sort_values(by='Promo Value',ascending=False) for key in globals()[f"modified_promotionBrands{slides_Period}_total"].keys()   if all(cat != key.split(' | ')[0] for cat in categories)}

In [172]:
for key in promotionsBrandsSelected:
    grand_total_row = promotionsBrandsSelected[key].loc[promotionsBrandsSelected[key]['Top Brands'] == 'Grand Total']
    sorted_df = promotionsBrandsSelected[key].loc[promotionsBrandsSelected[key]['Top Brands'] != 'Grand Total']
    promotionsBrandsSelected[key] = pd.concat([grand_total_row, sorted_df], ignore_index=True)

In [173]:
selectedBrands = selectedBrands_og

In [174]:
# Not including client brands
promotionsNotBrandsSelected = {
    key: globals()[f"modified_promotionBrands{slides_Period}_total"][key][
        ~globals()[f"modified_promotionBrands{slides_Period}_total"][key]['Top Brands'].isin(selectedBrands)
    ].sort_values(by='Value Share', ascending=False)
    for key in globals()[f"modified_promotionBrands{slides_Period}_total"].keys()
    if all(cat != key.split(' | ')[0] for cat in categories)
}


In [175]:
promotionsBrandsWithMarket=concatAttribute(promotionsBrandsSelected,marketList)
promotionsBrandsWithMarket = fillingMissingBrands(promotionsBrandsWithMarket)
promotionsNotBrandsWithMarket=concatAttribute(promotionsNotBrandsSelected,marketList)
promotionsNotBrandsWithMarket = fillingMissingBrands(promotionsNotBrandsWithMarket)

In [176]:
def split_market(data, Scope):
    final = {}
    for k,df in data.items():
        for key, value in Scope.items():
            df_market = df[df['SOURCE'].isin(value)]
            df_market = df_market.reset_index(drop=True)
            if df_market.shape[0] >0:
                final[k + ' | ' + value[0]] = df_market
    return final        

In [177]:
newpromotionsBrandsWithMarket = split_market(promotionsBrandsWithMarket,Scope)
newpromotionsNotBrandsWithMarket = split_market(promotionsNotBrandsWithMarket,Scope)

In [178]:
def concatAttributeNew(sorted):
    """
    Concatenate DataFrames from a sorted dictionary based on exact matches of categories, sectors, segments,
    subsegments, and subcategories. Adds a 'SOURCE' column to each DataFrame indicating its market.

    Parameters:
    sorted (dict): Dictionary with keys like 'category | sector | segment | brand' and values as DataFrames.
    categories, sectors, segments, subsegments, subcategories (list): Lists of strings to match exactly.

    Returns:
    dict: Dictionary with matched group names as keys and concatenated DataFrames as values.
    """
    # Combine all lists and preserve order without duplicates
    lis = list(dict.fromkeys(categories + sectors + segments + subsegments + subcategories))

    marketDic = defaultdict(list)
    concatenatedDic = {}

    for i in lis:
        for key, df in sorted.items():
            parts = key.split(' | ')
            if i in parts:
                # Determine market label
        
                market_label = parts[1]  # category

                df = df.copy()  # Avoid modifying original
                df['SOURCE'] = market_label
                marketDic[i].append(df)

        if marketDic[i]:
            concatenatedDic[i] = pd.concat(marketDic[i], ignore_index=True)

    return concatenatedDic



In [179]:
concated = concatAttributeNew(globals()[f"modified_promotionBrands{slides_Period}"])

In [180]:
concated1 = concatAttributeNew(globals()[f"modified_promotionBrands_share{slides_Period}"])

In [181]:
globals()[f"new_modified_promotionProducts{slides_Period}"] = filter_data(globals()[f"modified_promotionProducts{slides_Period}"])

In [182]:
top20clientonly = filter_data_Top(globals()[f"modified_promotionProducts{slides_Period}"])
bottom20clientonly = filter_data_Bot(globals()[f"modified_promotionProducts{slides_Period}"])

In [183]:
def promotionsEndOfWeekCleaning(promotions_EndOfWeek, notInScope, col='Top Brands'):
    promotionsEndOfWeek = {}
    for key, value in promotions_EndOfWeek.items():
        df = value.copy()
        if df.shape[0] != 0:
            modified_key = key
            flag = False if any(element in modified_key for element in notInScope) else True
            if flag:
                promotionsEndOfWeek[modified_key] = df[df[col] != 'Grand Total'].reset_index(drop=True).replace(np.nan, 0)
        else:
            print(key, ' Is empty')
    
    return promotionsEndOfWeek


In [184]:
mod=cleaningData(promotions_EndOfWeek)
promotionsEndOfWeekCleaned=promotionsEndOfWeekCleaning(mod,notInScope,col='End of Week')


In [185]:

brandMarketCategory = [key for key in promotionsEndOfWeekCleaned.keys() if any(cat in key.split(' | ')[0] for cat in categories )]
if len(sectors) != 0:
    brandMarketSector = [key for key in promotionsEndOfWeekCleaned.keys() if any(cat == key.split(' | ')[0] for cat in sectors )]
if len(segments) != 0:
    brandMarketSegment = [key for key in promotionsEndOfWeekCleaned.keys() if any(cat == key.split(' | ')[0] for cat in segments )]
if len(subsegments) != 0:
    brandMarketSubSegment = [key for key in promotionsEndOfWeekCleaned.keys() if any(cat == key.split(' | ')[0] for cat in subsegments )]
if len(subcategories) != 0:
    brandMarketSubCategory = [key for key in promotionsEndOfWeekCleaned.keys() if any(cat == key.split(' | ')[0] for cat in subcategories )]


In [186]:
def completeDates(dfList, promotionsEndOfWeekCleaned):
    # Create a list of unique brand-category combinations
    brandCatList = sorted(set(key.split(' | ')[0] + ' | ' + key.split(' | ')[1] for key in dfList))
    EndOfWeekcompletDate = {}
    dfGroup = []
    dic = defaultdict(int)
    for key in brandCatList:
        for name in dfList:
            if (key.split(' | ')[0] == name.split(' | ')[0]) and (key.split(' | ')[1] == name.split(' | ')[1]):
                dic[key] += 1
                
    # Iterate over unique brand-category combinations
    for name in dic.keys():
        # Get dataframe keys associated with the current brand-category combination
        dfName = [key for key in dfList if name == (key.split(' | ')[0] + ' | ' + key.split(' | ')[1])]
        uniqueDates = pd.concat([promotionsEndOfWeekCleaned[key] for key in dfName])[['End of Week']].drop_duplicates()
        if uniqueDates.shape[0] > 0:
            dfCompleteDates = {}
            dfGroup.append(dfName)
            for key in dfName:
                EndOfWeekcompletDate[key] = pd.merge(uniqueDates, promotionsEndOfWeekCleaned[key], how='left').replace(np.nan, 0).sort_values(by='End of Week').reset_index(drop = True)
    return EndOfWeekcompletDate, dfGroup, dic


In [187]:
if len(categories) != 0:
    dfCategory,catGroup,catDuplication=completeDates(brandMarketCategory,promotionsEndOfWeekCleaned)
if len(sectors) != 0:
    dfSector,secGroup,secDuplication=completeDates(brandMarketSector,promotionsEndOfWeekCleaned)
if len(segments) != 0:
    dfSegment,segGroup,segDuplication=completeDates(brandMarketSegment,promotionsEndOfWeekCleaned)
if len(subsegments) != 0:
    dfSubSegment,subsegGroup,subsegDuplication=completeDates(brandMarketSubSegment,promotionsEndOfWeekCleaned)
if len(subcategories) != 0:
    dfSubCategory,subcatGroup,subcatDuplication=completeDates(brandMarketSubCategory,promotionsEndOfWeekCleaned)

In [188]:
if len(sectors):
    sect_vsod_count =0
    for key,df in sect_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                sect_vsod_count +=1
    sect_vsod_count = sect_vsod_count *len(categories)
 
if len(segments):
    seg_vsod_count =0
    for key,df in seg_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                seg_vsod_count +=1
    seg_vsod_count = seg_vsod_count           
 
if len(subsegments) >0:
    subseg_vsod_count =0
    for key,df in subseg_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                subseg_vsod_count +=1
    subseg_vsod_count = subseg_vsod_count
 
if len(subcategories) >0:
    subcat_vsod_count =0
    for key,df in subcat_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                subcat_vsod_count +=1
    subcat_vsod_count = subcat_vsod_count

In [189]:
PromoRet ={}
if len(categories)!=0:
    first_key, first_value = next(iter(catDuplication.items()))
    PromoRet.update({first_key: first_value})
if len(sectors)!=0:
    sec_key, sec_value = next(iter(secDuplication.items()))
    PromoRet.update({sec_key:sec_value})
if len(segments)!=0:
    third_key, third_value = next(iter(segDuplication.items()))
    PromoRet.update({third_key: third_value})
if len(subsegments)!=0:
    fourth_key, fourth_value = next(iter(subsegDuplication.items()))
    PromoRet.update({fourth_key:fourth_value})
if len(subcategories)!=0:
    fifth_key, fifth_value = next(iter(subcatDuplication.items()))
    PromoRet.update({fifth_key:fifth_value })

In [190]:
PromoSalesTypes_data = {}
for key in brands_promo_type.keys():
    df=brands_promo_type[key].copy()
    df["Promo Sales"] = pd.to_numeric(df["Promo Sales"], errors="coerce").fillna(0)
    df["Value Share"] = pd.to_numeric(df["Value Share"], errors="coerce").fillna(0)
    df = df[df['Promo Type'].notna()]
    brand_totals = df.groupby("Top Brands")['Promo Sales'].sum()
    df["Brand Total Sales"] = df["Top Brands"].map(brand_totals)
    df["% Promo Sales"] = df["Promo Sales"] / df["Brand Total Sales"]

    df = df[~df['Top Brands'].str.contains('Others|Grand Total', case=False)]
    df = df[df['Value Share'] > 0.01]
    df = df[df['Promo Sales'] > 0]
    # Select client brands and additional brands needed to make 10 brands
    df_client = selectClientBrands(brands_promo_type[key],'Top Brands', 'Value Share')
    comp_brand = df[~df['Top Brands'].isin(cb for cb in client_brands)].drop_duplicates("Top Brands")
    if not df_client.empty:
        comp_brand = comp_brand.nlargest(10-df_client["Top Brands"].nunique(), "Value Share")["Top Brands"].to_list()
        # Concatenate client brands and additional brands
        df = df[df["Top Brands"].isin(comp_brand + client_brands)]
        df = df.reset_index(drop=True)
        df = df.sort_values("Value Share", ascending=False).reset_index(drop=True)
        df = df[~df['Promo Type'].fillna('').str.contains('NONE/PL|Undefined|Nan', na=False)]
        # print(comp_brand)
        if df.shape[0]:
            PromoSalesTypes_data[key] =df

In [191]:
final_lis = []
cat_lis = []
if categories:
    for i in range(len(catGroup)):
        cat_lis += genrateIndexList(catGroup[i], chartIndex=14, chartCount=4)[0]
    final_lis.append(cat_lis)
else:
    final_lis.append([])

sec_lis = []
if sectors:
    for i in range(len(secGroup)):
        sec_lis += genrateIndexList(secGroup[i], chartIndex=14, chartCount=4)[0]
    final_lis.append(sec_lis)
else:
    final_lis.append([])

seg_lis=[]
if segments:
    for i in range(len(segGroup)):
        seg_lis += genrateIndexList(segGroup[i], chartIndex=14, chartCount=4)[0]
    final_lis.append(seg_lis)

else:
    final_lis.append([])

subseg_lis =[]
if subsegments:
    for i in range(len(subsegGroup)):
        subseg_lis +=  genrateIndexList(subsegGroup[i], chartIndex=14, chartCount=4)[0]
    final_lis.append(subseg_lis)
else:
    final_lis.append([])

subcat_lis =[]
if subcategories:
    for i in range(len(subcatGroup)):
        subcat_lis +=  genrateIndexList(subcatGroup[i], chartIndex=14, chartCount=4)[0]
    final_lis.append(subcat_lis)
else:
    final_lis.append([])

### New Slide 1

In [192]:
category_month_year=MonthYear_clean(Category_MonthYear,column='Category')
sector_month_year = MonthYear_clean(Sector_MonthYear,column='Sector')
segment_month_year = MonthYear_clean(Segment_MonthYear,column='Segment')
subcat_month_year = MonthYear_clean(SubCategory_MonthYear,column='SubCategory')
subseg_month_year = MonthYear_clean(SubSegment_MonthYear,column='SubSegment')

In [193]:
def split_month_year(data, column):
    final_month_year ={}
    for key,df in data.items():
        for sec in df[column].unique():
            newkey = key + ' | ' + sec
            new_df = df[df[column] == sec].reset_index(drop=True)
            if new_df.shape[0] > 0:
                final_month_year[newkey] = new_df
    return final_month_year

In [194]:
category_month_year1=split_month_year(category_month_year,'Category')
sector_month_year1 = split_month_year(sector_month_year,'Sector')
segment_month_year1 = split_month_year(segment_month_year,'Segment')
subseg_month_year1 = split_month_year(subseg_month_year,'SubSegment')
subcat_month_year1 = split_month_year(subcat_month_year,'SubCategory')

In [195]:
month_year1 = {}
month_year1.update(sector_month_year1)
month_year1.update(segment_month_year1)
month_year1.update(subcat_month_year1)
month_year1.update(subseg_month_year1)

In [196]:
brandMarketCategory = [key for key in category_month_year1.keys() if any(cat == key.split(' | ')[2]  for cat in categories )]
if len(sectors) != 0:
    brandMarketSector = [key for key in sector_month_year1.keys() if any(cat == key.split(' | ')[2]  for cat in sectors )]
if len(segments) != 0:
    brandMarketSegment = [key for key in segment_month_year1.keys() if any(cat == key.split(' | ')[2]  for cat in segments )]
if len(subsegments) != 0:
    brandMarketSubSegment = [key for key in subseg_month_year1.keys() if any(cat == key.split(' | ')[2] for cat in subsegments )]
if len(subcategories) != 0:
    brandMarketSubCategory = [key for key in subcat_month_year1.keys() if any(cat == key.split(' | ')[2] for cat in subcategories )]

In [197]:
def completeDates1(dfList, promotionsEndOfWeekCleaned,column="Sector"):
    brandCatList = sorted(set(key.split(' | ')[0]  for key in dfList))
    EndOfWeekcompletDate = {}
    dfGroup = []
    dic = defaultdict(int)
    for key in brandCatList:
        for name in dfList:
            if (key.split(' | ')[0] == name.split(' | ')[0]):
                
                dic[key] += 1
    for name in dic.keys():

        if column == "Sector" :
            dfName = [key for key in dfList if name == key.split(' | ')[0] and len(name.split(' | ')) == 1]
        else:
            dfName = [key for key in dfList if name == key.split(' | ')[0]  ]
        uniqueDates = pd.concat([promotionsEndOfWeekCleaned[key] for key in dfName])[['MonthYear']].drop_duplicates()
        dfCompleteDates = {}
        dfGroup.append(dfName)
        for key in dfName:
            EndOfWeekcompletDate[key] = pd.merge(uniqueDates, promotionsEndOfWeekCleaned[key], how='left')#.replace(np.nan, 0)
            column = EndOfWeekcompletDate[key].columns[1]
            year = EndOfWeekcompletDate[key].columns[3]
            monthyear = EndOfWeekcompletDate[key].columns[0]
            EndOfWeekcompletDate[key][column] = EndOfWeekcompletDate[key][column].fillna(method='ffill')      
            EndOfWeekcompletDate[key][year] = pd.to_datetime(EndOfWeekcompletDate[key][monthyear], format='%b-%y').dt.year
            EndOfWeekcompletDate[key] = EndOfWeekcompletDate[key].fillna(0)
    return EndOfWeekcompletDate, dfGroup, dic

In [198]:
dfCategory0,categoryGroup0,categoryDuplication0=completeDates1(brandMarketCategory,category_month_year1,column="Category")
if len(sectors) != 0:
    dfSector0,secGroup0,secDuplication0=completeDates1(brandMarketSector,sector_month_year1,column="Sector")
if len(segments) != 0:
    dfSegment0,segGroup0,segDuplication0=completeDates1(brandMarketSegment,segment_month_year1,column="Segment")
if len(subsegments) != 0:
    dfSubSegment0,subsegGroup0,subsegDuplication0=completeDates1(brandMarketSubSegment,subseg_month_year1,column="Subsegment")
if len(subcategories) != 0:
    dfSubCategory0,subcatGroup0,subcatDuplication0=completeDates1(brandMarketSubCategory,subcat_month_year1,column="Subcategory")

In [199]:
categoryGroup0=groupingkeys(categoryGroup0)
if len(sectors) != 0:
    secGroup0=groupingkeys(secGroup0)
if len(segments) != 0:
    segGroup0=groupingkeys(segGroup0)
if len(subsegments) != 0:
    subsegGroup0=groupingkeys(subsegGroup0)
if len(subcategories) != 0:
    subcatGroup0=groupingkeys(subcatGroup0)

In [200]:
final_lis0 = []
category_lis = []
if categories:
    for i in range(len(categoryGroup0)):
        category_lis += genrateIndexList(categoryGroup0[i], chartIndex=19, chartCount=6)[0]
final_lis0.append(category_lis)  # Append empty list if sectors is False

sec_lis = []
if sectors:
    for i in range(len(secGroup0)):
        sec_lis += genrateIndexList(secGroup0[i], chartIndex=19, chartCount=6)[0]
final_lis0.append(sec_lis)  # Append empty list if sectors is False
print(final_lis0)

seg_lis = []
if segments:
    for i in range(len(segGroup0)):
        seg_lis += genrateIndexList(segGroup0[i], chartIndex=19, chartCount=6)[0]
final_lis0.append(seg_lis)  # Append empty list if segments is False
print(final_lis0)

subseg_lis = []
if subsegments:
    for i in range(len(subsegGroup0)):
        subseg_lis += genrateIndexList(subsegGroup0[i], chartIndex=19, chartCount=6)[0]
final_lis0.append(subseg_lis)  # Append empty list if subsegments is False
print(final_lis0)

subcat_lis = []
if subcategories:
    for i in range(len(subcatGroup0)):
        subcat_lis += genrateIndexList(subcatGroup0[i], chartIndex=19, chartCount=6)[0]
final_lis0.append(subcat_lis)  # Append empty list if subcategories is False


[[20, 20, 20, 20, 20, 20, 20, 20], [21, 21, 21, 21, 21, 21, 21, 21]]
[[20, 20, 20, 20, 20, 20, 20, 20], [21, 21, 21, 21, 21, 21, 21, 21], [22, 22, 22, 22, 22, 22, 22, 22]]
[[20, 20, 20, 20, 20, 20, 20, 20], [21, 21, 21, 21, 21, 21, 21, 21], [22, 22, 22, 22, 22, 22, 22, 22], []]


### New slide 2

In [201]:
globals()[f"modified_endofweek_{slides_Period}"] = {}
past_12_months = pd.date_range(end=end_date , periods=12, freq='M').strftime('%b-%y').tolist()
for key in modified_promotionEndOfWeek.keys():
    df=modified_promotionEndOfWeek[key].copy()
    df['End of Week'] = pd.to_datetime(df['End of Week'])
    filtered_df = df[df['End of Week'].dt.strftime('%b-%y').isin(past_12_months)]
    if filtered_df.shape[0] >0:
        globals()[f"modified_endofweek_{slides_Period}"][key] = filtered_df 

In [202]:
brandMarketCategory= [key for key in globals()[f"modified_endofweek_{slides_Period}"].keys() if any(cat in key.split(' | ')[0] for cat in categories )]
if len(sectors) != 0:
    brandMarketSector = [key for key in globals()[f"modified_endofweek_{slides_Period}"].keys() if any(cat == key.split(' | ')[0] for cat in sectors )]
if len(segments) != 0:
    brandMarketSegment = [key for key in globals()[f"modified_endofweek_{slides_Period}"].keys() if any(cat == key.split(' | ')[0] for cat in segments )]
if len(subsegments) != 0:
    brandMarketSubSegment = [key for key in globals()[f"modified_endofweek_{slides_Period}"].keys() if any(cat == key.split(' | ')[0] for cat in subsegments )]
if len(subcategories) != 0:
    brandMarketSubCategory = [key for key in globals()[f"modified_endofweek_{slides_Period}"].keys() if any(cat == key.split(' | ')[0] for cat in subcategories )]

In [203]:
if len(categories) != 0:
    dfCategory1,catGroup1,catDuplication1=completeDates(brandMarketCategory,globals()[f"modified_endofweek_{slides_Period}"])
if len(sectors) != 0:
    dfSector1,secGroup1,secDuplication1=completeDates(brandMarketSector,globals()[f"modified_endofweek_{slides_Period}"])
if len(segments) != 0:
    dfSegment1,segGroup1,segDuplication1=completeDates(brandMarketSegment,globals()[f"modified_endofweek_{slides_Period}"])
if len(subsegments) != 0:
    dfSubSegment1,subsegGroup1,subsegDuplication1=completeDates(brandMarketSubSegment,globals()[f"modified_endofweek_{slides_Period}"])
if len(subcategories) != 0:
    dfSubCategory1,subcatGroup1,subcatDuplication1=completeDates(brandMarketSubCategory,globals()[f"modified_endofweek_{slides_Period}"])

In [204]:
def promofrequencyclean(data):        
        modified_dfCategory1 = {}
        for k in data.keys():
                chart_df=data[k].copy()
                chart_df['Weekly VSOD'] = np.where((chart_df['VSOD']>.2)&(chart_df['Value Uplift (v. base) Normalized'] != ''),1,None)
                chart_df['try'] = 0
                chart_df['New Uplift'] = 0
                chart_df['try'] = np.where((chart_df['Value Uplift (v. base) Normalized']>=2),1.8,chart_df['Value Uplift (v. base) Normalized'])
                chart_df['New Uplift'] = np.where((chart_df['Weekly VSOD']==1)&(chart_df['Value Uplift (v. base) Normalized']>0.05),chart_df['try'],None)
                if not chart_df['Weekly VSOD'].isnull().all():
                        modified_dfCategory1[k]= chart_df 
        return modified_dfCategory1

In [205]:
if len(categories)!=0: 
    modified_dfCategory1=promofrequencyclean(dfCategory1)
if len(sectors)!=0: 
    modified_dfSector1=promofrequencyclean(dfSector1)
if len(segments)!=0: 
    modified_dfSegment1=promofrequencyclean(dfSegment1)
if len(subsegments)!=0: 
    modified_dfSubSegment1=promofrequencyclean(dfSubSegment1)
if len(subcategories)!=0: 
    modified_dfSubCategory1=promofrequencyclean(dfSubCategory1)

In [206]:
brandMarketCategory= [key for key in modified_dfCategory1.keys() if any(cat in key.split(' | ')[0] for cat in categories )]
if len(sectors) != 0:
    brandMarketSector = [key for key in modified_dfSector1.keys() if any(cat == key.split(' | ')[0] for cat in sectors )]
if len(segments) != 0:
    brandMarketSegment = [key for key in modified_dfSegment1.keys() if any(cat == key.split(' | ')[0] for cat in segments )]
if len(subsegments) != 0:
    brandMarketSubSegment = [key for key in modified_dfSubSegment1.keys() if any(cat == key.split(' | ')[0] for cat in subsegments )]
if len(subcategories) != 0:
    brandMarketSubCategory = [key for key in modified_dfSubCategory1.keys() if any(cat == key.split(' | ')[0] for cat in subcategories )]

In [207]:
if len(categories) != 0:
    dfCategory1,catGroup1,catDuplication1=completeDates(brandMarketCategory,globals()[f"modified_endofweek_{slides_Period}"])
if len(sectors) != 0:
    dfSector1,secGroup1,secDuplication1=completeDates(brandMarketSector,globals()[f"modified_endofweek_{slides_Period}"])
if len(segments) != 0:
    dfSegment1,segGroup1,segDuplication1=completeDates(brandMarketSegment,globals()[f"modified_endofweek_{slides_Period}"])
if len(subsegments) != 0:
    dfSubSegment1,subsegGroup1,subsegDuplication1=completeDates(brandMarketSubSegment,globals()[f"modified_endofweek_{slides_Period}"])
if len(subcategories) != 0:
    dfSubCategory1,subcatGroup1,subcatDuplication1=completeDates(brandMarketSubCategory,globals()[f"modified_endofweek_{slides_Period}"])

In [208]:
def group_by_region(data):
    # Define categories for grouping
    market_groups = {
        "RETAILER_REGIONS": regions_RET,
        "RETAILER_CHANNELS": channels_RET,
        "RETAILER_MARKET": market_RET,
        "CHANNEL_REGIONS": regions_CHAN,
        "CHANNEL_CHANNELS": channels_CHAN,
        "CHANNEL_MARKET": market_CHAN,
        f"{customareas}_REGIONS": regions_CUST,
        f"{customareas}_CHANNELS": channels_CUST,
        f"{customareas}_MARKET": market_CUST,
    }
    result = []
    for sublist in data:
        for category, keywords in market_groups.items():
            # Filter items matching the current category
            base_category = category.split("_")[0]

            group = [
                f"{item} | {base_category}" for item in sublist if item.split(" | ")[-1] in keywords
            ]
            if group:  # Append only non-empty groups
                result.append(group)
    return result


In [209]:
if len(categories)>0:
    catGroup1 = group_by_region(catGroup1)
if len(sectors)>0:
    secGroup1 = group_by_region(secGroup1)
if len(segments)>0:
    segGroup1 = group_by_region(segGroup1)
if len(subsegments)>0:
    subsegGroup1 = group_by_region(subsegGroup1)
if len(subcategories)>0:
    subcatGroup1 = group_by_region(subcatGroup1)

In [210]:
final_lis1 = []
cat_lis = []
if categories:
    for i in range(len(catGroup1)):
        cat_lis += genrateIndexList(catGroup1[i], chartIndex=25, chartCount=4)[0]
    final_lis1.append(cat_lis)
else:
    final_lis1.append([])

sec_lis = []
if sectors:
    for i in range(len(secGroup1)):
        sec_lis += genrateIndexList(secGroup1[i], chartIndex=25, chartCount=4)[0]
    final_lis1.append(sec_lis)
else:
    final_lis1.append([])

seg_lis=[]
if segments:
    for i in range(len(segGroup1)):
        seg_lis += genrateIndexList(segGroup1[i], chartIndex=25, chartCount=4)[0]
    final_lis1.append(seg_lis)

else:
    final_lis1.append([])

subseg_lis =[]
if subsegments:
    for i in range(len(subsegGroup1)):
        subseg_lis +=  genrateIndexList(subsegGroup1[i], chartIndex=25, chartCount=4)[0]
    final_lis1.append(subseg_lis)
else:
    final_lis1.append([])

subcat_lis =[]
if subcategories:
    for i in range(len(subcatGroup1)):
        subcat_lis +=  genrateIndexList(subcatGroup1[i], chartIndex=25, chartCount=4)[0]
    final_lis1.append(subcat_lis)
else:
    final_lis1.append([])

In [211]:
retailer=regions_RET+channels_RET+market_RET
channels=regions_CHAN+channels_CHAN+channels_CHAN
customarea=regions_CUST+channels_CUST+market_CUST
def addarea(modified_dfCategory1,retailer,market="RETAILER"):
    keys_to_modify = [k for k in modified_dfCategory1.keys() if k.split(" | ")[-1] in retailer]
    for k in keys_to_modify:
        new_key = k + " | "+ market
        modified_dfCategory1[new_key] = modified_dfCategory1[k]  
        del modified_dfCategory1[k]       
    return modified_dfCategory1      
if len(categories)>0:            
    modified_dfCategory1=addarea(modified_dfCategory1,retailer,market="RETAILER")
    modified_dfCategory1=addarea(modified_dfCategory1,channels,market="CHANNELS")
    modified_dfCategory1=addarea(modified_dfCategory1,customarea,market=f"{customareas}")

if len(sectors)>0:            
    modified_dfSector1=addarea(modified_dfSector1,retailer,market="RETAILER")
    modified_dfSector1=addarea(modified_dfSector1,channels,market="CHANNELS")
    modified_dfSector1=addarea(modified_dfSector1,customarea,market=f"{customareas}")
if len(segments)>0:            
    modified_dfSegment1=addarea(modified_dfSegment1,retailer,market="RETAILER")
    modified_dfSegment1=addarea(modified_dfSegment1,channels,market="CHANNELS")
    modified_dfSegment1=addarea(modified_dfSegment1,customarea,market=f"{customareas}")
if len(subsegments)>0:            
    modified_dfSubSegment1=addarea(modified_dfSubSegment1,retailer,market="RETAILER")
    modified_dfSubSegment1=addarea(modified_dfSubSegment1,channels,market="CHANNELS")
    modified_dfSubSegment1=addarea(modified_dfSubSegment1,customarea,market=f"{customareas}")
if len(subcategories)>0:            
    modified_dfSubCategory1=addarea(modified_dfSubCategory1,retailer,market="RETAILER")
    modified_dfSubCategory1=addarea(modified_dfSubCategory1,channels,market="CHANNELS")
    modified_dfSubCategory1=addarea(modified_dfSubCategory1,customarea,market=f"{customareas}")


In [212]:
for k in list(globals()[f"modified_promotionBrands{slides_Period}"].keys()):  # Convert to list to avoid runtime errors
    df = globals()[f"modified_promotionBrands{slides_Period}"][k].copy()
    # Filter rows based on 'Top Brands'
    df = df[~df['Top Brands'].str.contains('Others', case=False, na=False)]
    df = df[~df['Top Brands'].str.contains('Grand Total', case=False, na=False)]
    df = df[df['Value Share'] > 0.01]
    if not df.empty:
        globals()[f"modified_promotionBrands{slides_Period}"][k] = df
    else:
        del globals()[f"modified_promotionBrands{slides_Period}"][k]


In [213]:
for k in list(globals()[f"modified_promotionBrands_share{slides_Period}"].keys()):  # Convert to list to avoid runtime errors
    df = globals()[f"modified_promotionBrands_share{slides_Period}"][k].copy()
    # Filter rows based on 'Top Brands'
    df = df[~df['Top Brands'].str.contains('Others', case=False, na=False)]
    df = df[~df['Top Brands'].str.contains('Grand Total', case=False, na=False)]
    df = df[df['Value Share'] > 0.01]
    if not df.empty:
        globals()[f"modified_promotionBrands_share{slides_Period}"][k] = df
    else:
        del globals()[f"modified_promotionBrands_share{slides_Period}"][k]


## Slide duplication: index, duplication and section names

In [214]:
index = [*[0]*5,
         #*[1]*5,
         *[1]*5,
         *[2]*5,
         *[3]*5,
         *[4]*5,
         *[5]*5,
         *[6]*5,
         *[7]*5,
         *[8]*4,
         *[9]*5,
         *[10]*5,
         *[11]*5,
         *[12]*5,
         *[13]*5,
         *[14]*5,
         *final_lis,
         *[19]*5,
         *final_lis0,
         *final_lis1,
         *[0]*5,
         *[1]*5,
         *[2]*5,
         *[9]*5,
         *[10]*5,
        #  *[11]*5,
         *[12]*5,
         *[13]*5
         #*[14]*5
        ]
duplication = combine_duplications(Scope,count_df,[#modified_promotionBrandsP12M, #0
                                                   promotionsBrandSortedTotalFinal, #1
                                                   newpromotionsNotBrandsWithMarket, #2
                                                   concated, #3
                                                   globals()[f"modified_promotionProducts{slides_Period}_volumeuplift"], #4
                                                   globals()[f"new_modified_promotionProducts{slides_Period}"], #5
                                                   globals()[f"new_modified_promotionProducts{slides_Period}"], #6
                                                   top20clientonly, #7
                                                   bottom20clientonly,#8
                                                   globals()[f"modified_promotionBrands{slides_Period}"], #10
                                                   newModifiedBrands, #11
                                                   PromoSalesTypes_data if promo_type else None,#12
                                                   globals()[f"modified_promotionBrands_share{slides_Period}"] if feature_share else None, #13
                                                   globals()[f"modified_promotionBrands_share{slides_Period}"] if display_share else None, #14
                                                   modified_promotionEndOfWeek,#15
                                                   PromoRet, #16-19
                                                   modified_valueUplift, #20
                                                   #month_year1,#21
                                                   #modified_endofweek_P12M, #22
                                                   #modified_promotionBrandsP12M, #0 with no client
                                                   promotionsBrandNOTSortedTotalFinal, #1 with no client
                                                   newpromotionsNotBrandsWithMarket, #2 with no client
                                                   concated, #3 with no client
                                                   globals()[f"modified_promotionBrands{slides_Period}"], # 10 with no client
                                                   newModifiedBrands, #11 with no client
                                                   #PromoSalesTypes_data if promo_type else None,#12  with no client
                                                   globals()[f"modified_promotionBrands_share{slides_Period}"] if feature_share else None, #13  with no client
                                                   globals()[f"modified_promotionBrands_share{slides_Period}"] if display_share else None #14 with no client
                                                  ])
section_names = [#"Promo Value Sale",#0
                 "Promo Evolution", #1
                 "VSOD Summary by Sector" , #2
                 "Value uplift by retailer by brand", #3 
                 "Volume Uplift vs discount depth",#4
                 "Value Uplift vs Promo Efficiency Quadrant", #5
                 "Top 20 promotions", #6
                 "Top 20 promotions CLIENT ONLY", #7
                 "Bottom 20 promotions CLIENT ONLY", #8
                 "Promo share vs Value Share", #10
                 "Promo Sales by total size",#11
                 "Promo Sales by promo type", #12
                 "Feature Share vs Fair Share", #13
                 "Display Share vs Fair Share", #14
                 "Promo Frequency learnings", #15
                 "Promo sales per retailer", #16-19
                 "Value Uplift vs discount depth" ,#20
                 #"Seasonality Index",#21
                 #"Promotional Frequency Analysis", #22
                 #"Promo Value Sale no client prio",
                 "Promo Evolution no client prio",
                 "VSOD Summary by Sector no client prio",
                 "Value uplift by retailer by brand no client prio",
                "Promo share vs Value Share no client prio", #10
                 "Promo Sales by total size no client prio",#11
                 #"Promo Sales by promo type no client prio", #12
                 "Feature Share vs Fair Share no client prio", #13
                 "Display Share vs Fair Share no client prio" #14
                ]



#duplication.insert(89, 0)

if len(sectors) > 0:
       #duplication.insert(45,(len(client_manuf)+len(client_brands))*len(categories)* len(marketList))
       duplication.insert(40, sect_vsod_count)
if len(segments) > 0:
        #duplication.insert(46,(len(client_manuf)+len(client_brands))*len(sectors)* len(marketList)) 
         duplication.insert(41, seg_vsod_count)
 
else:
    duplication.insert(41,0)  
  
if len(subsegments) > 0:
        #duplication.insert(47,(len(client_manuf)+len(client_brands))*len(segments)* len(marketList))
        duplication.insert(42, subseg_vsod_count)

else:
    duplication.insert(42,0)

if len(subcategories) > 0:
        #duplication.insert(48,(len(client_manuf)+len(client_brands))*len(segments)* len(marketList))
        duplication.insert(43, subcat_vsod_count)

else:
    duplication.insert(43,0)


duplication.insert(84,1)
duplication.insert(85, 1)
duplication.insert(86, 1)
duplication.insert(87, 1)
duplication.insert(88, 0)
duplication.insert(89, 1)
duplication.insert(90, 1)
duplication.insert(91, 1)
duplication.insert(92, 1)
duplication.insert(93, 0)

section_names = [f"{name} {suffix}" for name in section_names for suffix in suffixes]

section_names.insert(40,'Volume Sold on Deal Sector')
section_names.insert(41,'Volume Sold on Deal Segment')
section_names.insert(42,'Volume Sold on Deal SubSegment')
section_names.insert(43,'Volume Sold on Deal SubCategory')

section_names.insert(84,'Seasonality Index Category')
section_names.insert(85,'Seasonality Index Sector')
section_names.insert(86,'Seasonality Index Segment')
section_names.insert(87,'Seasonality Index Subsegment')
section_names.insert(88,'Seasonality Index Subcategory')

section_names.insert(89,'Promotional Frequency Analysis Category')
section_names.insert(90,'Promotional Frequency Analysis Sector')
section_names.insert(91,'Promotional Frequency Analysis Segment')
section_names.insert(92,'Promotional Frequency Analysis Subsegment')
section_names.insert(93,'Promotional Frequency Analysis Subcategory')
#section_names.insert(94,'Promotional Frequency Analysis Subcategory')

duplication[77]=1
#index = [i for i in index if i != []]
# duplication = [i for i in duplication if i != []]
#

path = os.getcwd() + '//Promotion base.pptx'
new_pre = os.getcwd() + '//slide duplicated.pptx'


### Deck 1

In [215]:
print(index)
print(duplication)
print(section_names)
print(len(index))
print(len(duplication))
print(len(section_names))
print(final_lis)
print(final_lis1)
print(sum(duplication))

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, [18, 18, 18, 18, 18, 18, 18, 18], [18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18], [18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 16, 18, 18, 18, 18], [], [], 19, 19, 19, 19, 19, [20, 20, 20, 20, 20, 20, 20, 20], [21, 21, 21, 21, 21, 21, 21, 21], [22, 22, 22, 22, 22, 22, 22, 22], [], [], [27, 29, 26, 27, 29, 26, 27, 29, 26, 27, 29, 26], [27, 29, 26, 27, 29, 26, 27, 29, 26, 27, 29, 26, 27, 29, 26, 27, 29, 26], [27, 29, 26, 27, 29, 26, 27, 29, 26, 27, 29, 26, 27, 29, 26, 27, 29, 27, 29, 26, 27, 29, 26], [], [], 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13]
[7, 14, 21, 0, 0, 0, 8, 8, 0, 0, 1, 2, 3, 0, 0, 7, 14, 21, 0, 0, 7, 14, 19, 0, 0, 7, 14, 19, 0, 0, 7, 14, 18, 0, 0

In [216]:
slideDuplication(index,duplication,section_names,path,new_pre)

KeyboardInterrupt: 

## Replace duplicated slides with data

In [ ]:
prs = Presentation(new_pre)
posItr = 0
ind =0

In [ ]:
# slide 2
for key,value in Scope.items():
    dict = {key: count_df(promotionsBrandSortedTotalFinal,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in promotionsBrandSortedTotalFinal.items() if key in dict[key1]}
        if filtered_dict:
            promoEvolutionNew(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1


In [ ]:
print(ind,posItr)

5 42


In [ ]:
# slide 3
for key,value in Scope.items():
    dict = {key: count_df(newpromotionsBrandsWithMarket,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in newpromotionsBrandsWithMarket.items() if key in dict[key1]}
        if filtered_dict:
            VSOD1(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(ind,posItr)

10 58


In [ ]:
# slide 4
for key,value in Scope.items():
    dict = {key: count_df(concated,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in concated.items() if key in dict[key1]}
        if filtered_dict:
            valueUpliftRetailer(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(ind,posItr)

15 64


In [ ]:
# slide 5
for key,value in Scope.items():
    dict = {key: count_df(globals()[f"modified_promotionProducts{slides_Period}_volumeuplift"],value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in globals()[f"modified_promotionProducts{slides_Period}_volumeuplift"].items() if key in dict[key1]}
        if filtered_dict:
            VolumeUplift(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:

print(ind,posItr)

20 106


In [ ]:
# slide 6
for key,value in Scope.items():
    dict = {key: count_df(globals()[f"new_modified_promotionProducts{slides_Period}"],value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in globals()[f"new_modified_promotionProducts{slides_Period}"].items() if key in dict[key1]}
        if filtered_dict:
            ValueUpliftvsPromoEfficiencyQuadrant(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(ind,posItr)

25 146


In [ ]:
# slide 7
for key,value in Scope.items():
    dict = {key: count_df(globals()[f"new_modified_promotionProducts{slides_Period}"],value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in globals()[f"new_modified_promotionProducts{slides_Period}"].items() if key in dict[key1]}
        if filtered_dict:
            top20(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(ind,posItr)

30 186


In [ ]:
# slide 8
for key,value in Scope.items():
    dict = {key: count_df(top20clientonly,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in top20clientonly.items() if key in dict[key1]}
        if filtered_dict:
            top20Client(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(ind,posItr)

35 225


In [ ]:
# slide 9
for key,value in Scope.items():
    dict = {key: count_df(bottom20clientonly,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in bottom20clientonly.items() if key in dict[key1]}
        if filtered_dict:
            bot20Client(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(ind,posItr)

40 264


In [ ]:
if len(sectors)>0:
    newVolumeSold(prs, sect_vsod_merged, position=posItr, parent=direct_parent['Sector'], child = 'Sector')
    print(posItr)
    posItr += sect_vsod_count
    ind +=1
else:
    ind+=1

264


In [ ]:
print(ind,posItr)

41 292


In [ ]:
if len(segments)>0:
    newVolumeSold(prs, seg_vsod_merged, position=posItr, parent=direct_parent['Segment'], child = 'Segment')
    posItr += seg_vsod_count
    ind +=1
    
else:
    ind+=1

In [ ]:
print(ind,posItr)

42 348


In [ ]:
if len(subsegments)>0:
    newVolumeSold(prs, subseg_vsod_merged, position=posItr, parent=direct_parent['SubSegment'], child = 'SubSegment')
    posItr += subseg_vsod_count
    ind+=1
else:
    ind+=1

In [ ]:
print(ind,posItr)

43 348


In [ ]:
if len(subcategories)>0:
    newVolumeSold(prs, subcat_vsod_merged, position=posItr, parent=direct_parent['SubCategory'], child = 'SubCategory')
    posItr += subcat_vsod_count
    ind+=1
else:
    ind+=1

In [ ]:

print(ind,posItr)

44 348


In [ ]:
# slide 11
for key,value in Scope.items():
    dict = {key: count_df(globals()[f"modified_promotionBrands{slides_Period}"],value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in globals()[f"modified_promotionBrands{slides_Period}"].items() if key in dict[key1]}
        if filtered_dict:
            PromoShare_vs_ValueShare(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(ind,posItr)

49 396


In [ ]:
# slide 12
for key,value in Scope.items():
    dict = {key: count_df(newModifiedBrands,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in newModifiedBrands.items() if key in dict[key1]}
        if filtered_dict:
            PromoSalesTotalSize(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(ind,posItr)

54 396


In [ ]:
# slide 13
if promo_type:
    for key,value in Scope.items():
        dict = {key: count_df(PromoSalesTypes_data,value) }
        for key1,value1 in dict.items():
            filtered_dict = {key: value for key, value in PromoSalesTypes_data.items() if key in dict[key1]}
            if filtered_dict:
                PromoSalesTypes(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
            posItr += len(filtered_dict)
            ind +=1
else:
    ind +=5

In [ ]:
print(ind,posItr)

59 396


In [ ]:
# slide 14
if feature_share:
    for key,value in Scope.items():
        dict = {key: count_df(globals()[f"modified_promotionBrands_share{slides_Period}"],value) }
        for key1,value1 in dict.items():
            filtered_dict = {key: value for key, value in globals()[f"modified_promotionBrands_share{slides_Period}"].items() if key in dict[key1]}
            if filtered_dict:    
                featureShare(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
            posItr += len(filtered_dict)
            ind +=1
else:
    ind +=5

In [ ]:
print(ind,posItr)

64 396


In [ ]:

# slide 15
if display_share:
    for key,value in Scope.items():
        dict = {key: count_df(globals()[f"modified_promotionBrands_share{slides_Period}"],value) }
        for key1,value1 in dict.items():
            filtered_dict = {key: value for key, value in globals()[f"modified_promotionBrands_share{slides_Period}"].items() if key in dict[key1]}
            if filtered_dict:    
                displayShare(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
            posItr += len(filtered_dict)
            ind +=1
            print(ind,posItr)
            
else:
    ind +=5

65 396
66 396
67 396
68 396
69 396


In [ ]:
print(ind,posItr)

69 396


In [ ]:
# slide 16
for key,value in Scope.items():
    dict = {key: count_df(modified_promotionEndOfWeek,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in modified_promotionEndOfWeek.items() if key in dict[key1]}
        if filtered_dict:
            PromoFrequency(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(ind,posItr)

74 542


In [ ]:
if categories:
    catFinal = sorted(splitDfsPromo(dfCategory,(client_manuf) ,genrateIndexList(catGroup[0])[0]))
    catFinal = catFinal+sorted(splitDfsPromo(dfCategory,(client_brands) ,genrateIndexList(catGroup[0])[0]))
    catFinal

In [ ]:
print(ind,posItr)

74 542


In [ ]:
if sectors:
    secFinal = sorted(splitDfsPromo(dfSector,(client_manuf)  ,genrateIndexList(secGroup[0])[0]))
    secFinal = secFinal + sorted(splitDfsPromo(dfSector,(client_brands)  ,genrateIndexList(secGroup[0])[0]))
    secFinal

In [ ]:
print(ind,posItr)

74 542


In [ ]:
if segments:
    segFinal = sorted(splitDfsPromo(dfSegment,(client_manuf)  ,genrateIndexList(segGroup[0])[0]))
    segFinal = segFinal+sorted(splitDfsPromo(dfSegment,(client_brands)  ,genrateIndexList(segGroup[0])[0]))
    segFinal

In [ ]:
if subsegments:
    subsegFinal = sorted(splitDfsPromo(dfSubSegment,(client_manuf)  ,genrateIndexList(subsegGroup[0])[0]))
    subsegFinal = subsegFinal + sorted(splitDfsPromo(dfSubSegment,(client_brands)  ,genrateIndexList(subsegGroup[0])[0]))
    subsegFinal

In [ ]:
if subcategories:
    subcatFinal = sorted(splitDfsPromo(dfSubCategory,(client_manuf) ,genrateIndexList(subcatGroup[0])[0]))
    subcatFinal = subcatFinal+sorted(splitDfsPromo(dfSubCategory,(client_brands) ,genrateIndexList(subcatGroup[0])[0]))
    subcatFinal

In [ ]:
#Slide 17
#split catGroup into Lists depends on num of charts 
catGroupSplit = splitListpromo(dfCategory, catGroup, [i-14 for i in index[ind]])
print(catGroupSplit)
promoSalesPerRetailer(prs,dfCategory,len(index[ind]),catGroupSplit,position=sum(duplication[:ind]))
posItr = sum(duplication[:ind]) + len(index[ind])
ind+=1
print(len(index[ind]),ind, posItr)

#split secGroup into Lists depends on num of charts 
if len(sectors) != 0: 
    secGroupSplit = splitListpromo(dfSector, secGroup, [i-14 for i in index[ind]])
    promoSalesPerRetailer(prs,dfSector,len(index[ind]),secGroupSplit,position=posItr)
    posItr += len(index[ind])
ind+=1
print(len(index[ind]),ind, posItr)

#split segGroup into Lists depends on num of charts 
if len(segments) != 0: 
    segGroupSplit = splitListpromo(dfSegment, segGroup, [i-14 for i in index[ind]])
    promoSalesPerRetailer(prs,dfSegment,len(index[ind]),segGroupSplit,position=posItr)
    posItr += len(index[ind])
ind+=1
print(len(index[ind]),ind, posItr)

#split subsegGroup into Lists depends on num of charts 
if len(subsegments) != 0:
    subsegGroupSplit = splitListpromo(dfSubSegment, subsegGroup, [i-14 for i in index[ind]])

    promoSalesPerRetailer(prs,dfSubSegment,len(index[ind]),subsegGroupSplit,position=posItr)
    posItr += len(index[ind])
ind+=1
print(len(index[ind]),ind, posItr)

#split subcatGroup into Lists depends on num of charts 
if len(subcategories) != 0:
    subcatGroupSplit = splitListpromo(dfSubCategory, subcatGroup, [i-14 for i in index[ind]])
    print(subcatGroupSplit)
    promoSalesPerRetailer(prs,dfSubCategory,len(index[ind]),subcatGroupSplit,position=posItr)
    posItr += len(index[ind])
ind+=1


[['Total Fromage | Bel | Carrefour', 'Total Fromage | Bel | Intermarche', 'Total Fromage | Bel | Carrefour Hyper + Drive', 'Total Fromage | Bel | Carrefour Supermarket + Drive'], ['Total Fromage | Bel | Carrefour Proximite', 'Total Fromage | Bel | Intermarche Super', 'Total Fromage | Bel | Intermarche Hyper', 'Total Fromage | Bel | Intermarche Proxi'], ['Total Fromage | Boursin | Carrefour', 'Total Fromage | Boursin | Intermarche', 'Total Fromage | Boursin | Carrefour Hyper + Drive', 'Total Fromage | Boursin | Carrefour Supermarket + Drive'], ['Total Fromage | Boursin | Carrefour Proximite', 'Total Fromage | Boursin | Intermarche Super', 'Total Fromage | Boursin | Intermarche Hyper', 'Total Fromage | Boursin | Intermarche Proxi'], ['Total Fromage | Kiri | Carrefour', 'Total Fromage | Kiri | Intermarche', 'Total Fromage | Kiri | Carrefour Hyper + Drive', 'Total Fromage | Kiri | Carrefour Supermarket + Drive'], ['Total Fromage | Kiri | Carrefour Proximite', 'Total Fromage | Kiri | Interm

In [ ]:
# slide 21
for key,value in Scope.items():
    dict = {key: count_df(modified_valueUplift,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in modified_valueUplift.items() if key in dict[key1]}
        if filtered_dict:
            valueUplift(prs,filtered_dict,duplication[ind],position=posItr)
        posItr += len(filtered_dict)
        print(ind,posItr)
        ind +=1


79 600
80 628
81 663
82 663
83 663


In [ ]:
print(ind, duplication[ind], posItr)

84 1 663


In [ ]:
if len(categories)>0:
    seasonality(prs,dfCategory0, len(index[ind]), categoryGroup0, position=posItr,slideby="Category")
    posItr += len(index[ind])
ind+=1
print(posItr, len(index[ind])) 

671 8


In [ ]:
if len(sectors)>0:
    seasonality(prs, dfSector0, len(index[ind]), secGroup0, position=posItr,slideby="Sector")
    posItr += len(index[ind])
ind+=1
print(posItr, len(index[ind])) 

679 8


In [ ]:
if len(segments)>0:
    seasonality(prs, dfSegment0, len(index[ind]), segGroup0, position=posItr,slideby="Segment")
    posItr += len(index[ind])
ind+=1
print(posItr, len(index[ind]),ind)  

687 0 87


In [ ]:
if len(subsegments) != 0:
    seasonality(prs,dfSubSegment0,len(index[ind]),subsegGroup0,position=posItr,slideby="SubSegment")
    posItr += len(index[ind])
ind+=1

print(posItr, len(index[ind]),ind)  

687 0 88


In [ ]:
if len(subcategories) != 0:
    seasonality(prs,dfSubCategory0,len(index[ind]),subcatGroup0,position=posItr,slideby="SubCategory")
    posItr += len(index[ind])
ind+=1


In [ ]:
print(len(index[ind]), ind, duplication[ind], posItr)

12 89 1 687


In [ ]:
print(posItr,ind)

687 89


In [ ]:
end_date1 = pd.to_datetime(end_date)
start_date1 = end_date1 - pd.DateOffset(months=12)
 
# Generate all weekly periods (weekly ends, e.g., Sundays) between start and end
week_ends = pd.date_range(start=start_date1, end=end_date1, freq='W-SUN')
 
# Convert to list of dates (if needed)
week_ends_list = week_ends.to_list()
all_weeks_df = pd.DataFrame({'End of Week': week_ends_list})

def add_all_weeks(data):
    final_data ={}
    for key,df in data.items():
        df_full = all_weeks_df.merge(df, on='End of Week', how='left')
        df_full.fillna(0, inplace=True)
        final_data[key] = df_full
    return final_data

In [ ]:
modified_dfCategory1 = add_all_weeks(modified_dfCategory1)
modified_dfSector1 = add_all_weeks(modified_dfSector1)
modified_dfSegment1 = add_all_weeks(modified_dfSegment1)

In [ ]:
catGroup1Split = splitListpromo(modified_dfCategory1, catGroup1, [i-25 for i in index[ind]])

Promotional_Frequency(prs,modified_dfCategory1,len(index[ind]),catGroup1Split,position=posItr)
posItr +=len(catGroup1Split)
ind+=1
print(len(index[ind]), posItr)
#Sector Replace
if len(sectors) != 0: 
    secGroup1Split = splitListpromo(modified_dfSector1, secGroup1, [i-25 for i in index[ind]])
    Promotional_Frequency(prs,modified_dfSector1,len(index[ind]),secGroup1Split,position=posItr)
    posItr += len(secGroup1Split)
ind+=1
print(len(index[ind]), posItr)


if len(segments) != 0: 
    segGroup1Split = splitListpromo(modified_dfSegment1, segGroup1, [i-25 for i in index[ind]])
    Promotional_Frequency(prs,modified_dfSegment1,len(index[ind]),segGroup1Split,position=posItr)
    posItr += len(segGroup1Split)
ind+=1
print(len(index[ind]), posItr)

if len(subsegments) != 0:
    subsegGroup1Split = splitListpromo(modified_dfSubSegment1, subsegGroup1, [i-25 for i in index[ind]])
    Promotional_Frequency(prs,modified_dfSubSegment1,len(index[ind]),subsegGroup1Split,position=posItr)
    posItr += len(subsegGroup1Split)
ind+=1
print(len(index[ind]), posItr)

if len(subcategories) != 0:
    subcatGroup1Split = splitListpromo(modified_dfSubCategory1, subcatGroup1, [i-25 for i in index[ind]])
    Promotional_Frequency(prs,modified_dfSubCategory1,len(index[ind]),subcatGroup1Split,position=posItr)
    posItr += len(subcatGroup1Split)
ind+=1


18 699
23 717
0 740
0 740


In [ ]:
print(posItr)

740


In [ ]:
# slide 2 with no client brands
for key,value in Scope.items():
    dict = {key: count_df(promotionsBrandNOTSortedTotalFinal,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in promotionsBrandNOTSortedTotalFinal.items() if key in dict[key1]}
        if filtered_dict:
            promoEvolutionNew(prs,filtered_dict,duplication[ind],position=posItr)
            posItr += len(filtered_dict)
        ind +=1


In [ ]:
print(posItr)

788


In [ ]:
# slide 3 with no client brands
for key,value in Scope.items():
    dict = {key: count_df(newpromotionsNotBrandsWithMarket,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in newpromotionsNotBrandsWithMarket.items() if key in dict[key1]}
        if filtered_dict:
            VSOD1(prs,filtered_dict,duplication[ind],position=posItr)
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(posItr)

804


In [ ]:
# slide 4 with no client brands
for key,value in Scope.items():
    dict = {key: count_df(concated,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in concated.items() if key in dict[key1]}
        if filtered_dict:
            valueUpliftRetailer_no(prs,filtered_dict,duplication[ind],position=posItr)
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(posItr)

810


In [ ]:
# slide 11 with no client prio
for key,value in Scope.items():
    dict = {key: count_df(globals()[f"modified_promotionBrands{slides_Period}"],value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in globals()[f"modified_promotionBrands{slides_Period}"].items() if key in dict[key1]}
        if filtered_dict:
            PromoShare_vs_ValueShare_no(prs,filtered_dict,duplication[ind],position=posItr)
        posItr += len(filtered_dict)
        ind +=1

In [ ]:
print(posItr)

858


In [ ]:
print(ind, posItr)


114 858


In [ ]:
# slide 12 with no client prio
for key,value in Scope.items():
    dict = {key: count_df(newModifiedBrands,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in newModifiedBrands.items() if key in dict[key1]}
        if filtered_dict:
            PromoSalesTotalSize_no(prs,filtered_dict,duplication[ind],position=posItr)
        posItr += len(filtered_dict)
    ind +=1
    print(posItr,ind)  
        

858 115
858 116
858 117
858 118
858 119


In [ ]:
# slide 14 with no client prio
if feature_share:
    for key,value in Scope.items():
        dict = {key: count_df(globals()[f"modified_promotionBrands_share{slides_Period}"],value) }
        for key1,value1 in dict.items():
            filtered_dict = {key: value for key, value in globals()[f"modified_promotionBrands_share{slides_Period}"].items() if key in dict[key1]}
            if filtered_dict:    
                featureShare_no(prs,filtered_dict,duplication[ind],position=posItr)
            posItr += len(filtered_dict)
            ind +=1
else:
    ind +=5
    

In [ ]:
# slide 15 with no client prio
if display_share:
    for key,value in Scope.items():
        dict = {key: count_df(globals()[f"modified_promotionBrands_share{slides_Period}"],value) }
        for key1,value1 in dict.items():
            filtered_dict = {key: value for key, value in globals()[f"modified_promotionBrands_share{slides_Period}"].items() if key in dict[key1]}
            if filtered_dict:    
                displayShare_no(prs,filtered_dict,duplication[ind],position=posItr)
            posItr += len(filtered_dict)
            ind +=1
else:
    ind +=5

## Output slides

In [ ]:
outputPath=os.getcwd() + f"\\Promotion {client_manuf[0]}.pptx"
prs.save(outputPath)
# app = win32.Dispatch("PowerPoint.Application")
final=os.getcwd() +f"\\Promotion {client_manuf[0]}.pptx"
open_chart_data_in_excel(final,outputPath)

Slide 59: Opened Excel workbook: Book1
Slide 60: Opened Excel workbook: Book1
Slide 61: Opened Excel workbook: Book1
Slide 62: Opened Excel workbook: Book1
Slide 63: Opened Excel workbook: Book1
Slide 64: Opened Excel workbook: Book1
Slide 65: Opened Excel workbook: Book1
Slide 66: Opened Excel workbook: Book1
Slide 67: Opened Excel workbook: Book1
Slide 68: Opened Excel workbook: Book1
Slide 69: Opened Excel workbook: Book1
Slide 70: Opened Excel workbook: Book1
Slide 71: Opened Excel workbook: Book1
Slide 72: Opened Excel workbook: Book1
Slide 73: Opened Excel workbook: Book1
Slide 74: Opened Excel workbook: Book1
Slide 75: Opened Excel workbook: Book1
Slide 76: Opened Excel workbook: Book1
Slide 77: Opened Excel workbook: Book1
Slide 78: Opened Excel workbook: Book1
Slide 79: Opened Excel workbook: Book1
Slide 80: Opened Excel workbook: Book1
Slide 81: Opened Excel workbook: Book1
Slide 82: Opened Excel workbook: Book1
Slide 83: Opened Excel workbook: Book1
Slide 84: Opened Excel wo